In [1]:
### Importing used libraries
import os
import math
import re
from decimal import Decimal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Data exploration

In [2]:
# Loading the train data 
train = pd.read_csv("auto-insurance-fall-2017/train_auto.csv")

In [3]:
### A look on our data
train.head(8)

INDEX  TARGET_FLAG  TARGET_AMT  KIDSDRIV   AGE  HOMEKIDS   YOJ    INCOME  \
0      1            0         0.0         0  60.0         0  11.0   $67,349   
1      2            0         0.0         0  43.0         0  11.0   $91,449   
2      4            0         0.0         0  35.0         1  10.0   $16,039   
3      5            0         0.0         0  51.0         0  14.0       NaN   
4      6            0         0.0         0  50.0         0   NaN  $114,986   
5      7            1      2946.0         0  34.0         1  12.0  $125,301   
6      8            0         0.0         0  54.0         0   NaN   $18,755   
7     11            1      4021.0         1  37.0         2   NaN  $107,961   

  PARENT1  HOME_VAL  ... BLUEBOOK TIF    CAR_TYPE RED_CAR  OLDCLAIM CLM_FREQ  \
0      No        $0  ...  $14,230  11     Minivan     yes    $4,461        2   
1      No  $257,252  ...  $14,940   1     Minivan     yes        $0        0   
2      No  $124,191  ...   $4,010   4       z_SUV      no   $38,690        2   
3      No  $306,251  ...  $15,440   7     Minivan     yes        $0        0   
4      No  $243,925  ...  $18,000   1       z_SUV      no   $19,217        2   
5     Yes        $0  ...  $17,430   1  Sports Car      no        $0        0   
6      No       NaN  ...   $8,780   1       z_SUV      no        $0        0   
7      No  $333,680  ...  $16,970   1         Van     yes    $2,374        1   

  REVOKED  MVR_PTS CAR_AGE           URBANICITY  
0      No        3    18.0  Highly Urban/ Urban  
1      No        0     1.0  Highly Urban/ Urban  
2      No        3    10.0  Highly Urban/ Urban  
3      No        0     6.0  Highly Urban/ Urban  
4     Yes        3    17.0  Highly Urban/ Urban  
5      No        0     7.0  Highly Urban/ Urban  
6      No        0     1.0  Highly Urban/ Urban  
7     Yes       10     7.0  Highly Urban/ Urban  

[8 rows x 26 columns]

In [4]:
### Data statistics
print('Our data contains %d instances and %d features.' % (train.shape[0],train.shape[1]-1))
print('Our data contains %d missing values.'%(train.isna().sum().sum()))

Our data contains 8161 instances and 25 features.
Our data contains 2405 missing values.


In [5]:
## Features type
train.dtypes

INDEX            int64
TARGET_FLAG      int64
TARGET_AMT     float64
KIDSDRIV         int64
AGE            float64
HOMEKIDS         int64
YOJ            float64
INCOME          object
PARENT1         object
HOME_VAL        object
MSTATUS         object
SEX             object
EDUCATION       object
JOB             object
TRAVTIME         int64
CAR_USE         object
BLUEBOOK        object
TIF              int64
CAR_TYPE        object
RED_CAR         object
OLDCLAIM        object
CLM_FREQ         int64
REVOKED         object
MVR_PTS          int64
CAR_AGE        float64
URBANICITY      object
dtype: object

In [6]:
### Missing values for each feature
train.isna().sum()

INDEX            0
TARGET_FLAG      0
TARGET_AMT       0
KIDSDRIV         0
AGE              6
HOMEKIDS         0
YOJ            454
INCOME         445
PARENT1          0
HOME_VAL       464
MSTATUS          0
SEX              0
EDUCATION        0
JOB            526
TRAVTIME         0
CAR_USE          0
BLUEBOOK         0
TIF              0
CAR_TYPE         0
RED_CAR          0
OLDCLAIM         0
CLM_FREQ         0
REVOKED          0
MVR_PTS          0
CAR_AGE        510
URBANICITY       0
dtype: int64

**We drop the index columns and rename TARGET_FLAG column as Target**

In [7]:
drop_features = ['INDEX','TARGET_AMT'] ### columns to drop
train.drop(columns=drop_features,inplace=True) ### droping columns
train.rename(columns={'TARGET_FLAG':'target'},inplace=True) ### renaming output

In [8]:
## now we will explore our target; we will see its values and if it is a balanced output or not
train.target.value_counts().plot(kind='bar', rot=0)
plt.show()

In [9]:
### classes propotions
train.target.value_counts()

0    6008
1    2153
Name: target, dtype: int64

**We observe that our data is unbalanced, we have 6008 instances of class 0 and 2153 for class 1, the problem is then an unbalanced classification.**

**In the next step, we will try to preprocess our data, this one contains a lot of categorical features and some missisng values along with the unbalanced output.**

# Preprocessing

**We will perform the same preprocessing on both train adn test daat so we will join them and then devide them after preocessing.**

In [10]:
test  = pd.read_csv("auto-insurance-fall-2017/test_auto.csv") 
test.drop(columns=drop_features,inplace=True) ### droping columns
test.rename(columns={'TARGET_FLAG':'target'},inplace=True) ### renaming output
data  =  train.append(test, ignore_index=True)

In [11]:
data.drop(columns=['target'],inplace=True)

### <font color='red'>Missing values</font>

**We will replace them by the most frequent values, its a simple imputation technique, we may use more advanced and accurate techniques such us knn imputation.**

In [12]:
### missing values 

conatins_nan_col = data.columns[np.where(data.isna().sum()!=0)[0]]

data[conatins_nan_col]=data[conatins_nan_col].fillna(data.mode().iloc[0])

In [13]:
#### check if there is any left missing values
data.isna().sum().sum()

0

**Perfect, no missing values left in our data move on to the next step !**

### <font color='red'>Feature engeneering</font>

**First let's divide our features into the following sets : integer features, float features, object features.**

In [14]:
integer_features = np.where((data.dtypes == np.int64))[0] # integer features
float_features   = np.where((data.dtypes == np.float64))[0]  #Float features
object_features  = np.where((data.dtypes != np.float64) & (data.dtypes != np.int64))[0]  # Object features

In [15]:
### a look on object features
data.iloc[:, object_features].head(8)

INCOME PARENT1  HOME_VAL MSTATUS  SEX      EDUCATION            JOB  \
0   $67,349      No        $0    z_No    M            PhD   Professional   
1   $91,449      No  $257,252    z_No    M  z_High School  z_Blue Collar   
2   $16,039      No  $124,191     Yes  z_F  z_High School       Clerical   
3        $0      No  $306,251     Yes    M   <High School  z_Blue Collar   
4  $114,986      No  $243,925     Yes  z_F            PhD         Doctor   
5  $125,301     Yes        $0    z_No  z_F      Bachelors  z_Blue Collar   
6   $18,755      No        $0     Yes  z_F   <High School  z_Blue Collar   
7  $107,961      No  $333,680     Yes    M      Bachelors  z_Blue Collar   

      CAR_USE BLUEBOOK    CAR_TYPE RED_CAR OLDCLAIM REVOKED  \
0     Private  $14,230     Minivan     yes   $4,461      No   
1  Commercial  $14,940     Minivan     yes       $0      No   
2     Private   $4,010       z_SUV      no  $38,690      No   
3     Private  $15,440     Minivan     yes       $0      No   
4     Private  $18,000       z_SUV      no  $19,217     Yes   
5  Commercial  $17,430  Sports Car      no       $0      No   
6     Private   $8,780       z_SUV      no       $0      No   
7  Commercial  $16,970         Van     yes   $2,374     Yes   

            URBANICITY  
0  Highly Urban/ Urban  
1  Highly Urban/ Urban  
2  Highly Urban/ Urban  
3  Highly Urban/ Urban  
4  Highly Urban/ Urban  
5  Highly Urban/ Urban  
6  Highly Urban/ Urban  
7  Highly Urban/ Urban

**We observe the following:**

- **We have some features representing an amount of money, so we can convert that them to float features, these features are: INCOME, HOME_VAL, BLUEBOOK and OLDCLAIM.**

- **Some features contains only two values 'yes' or 'No', we convert that to 0 and 1, these featreus are : PARENT1, MSTATUS, RED_CAR, REVOKED, SEX, CAR_USE and URBANICITY.**

In [16]:
### Converting money features to float
money_features = ['INCOME', 'HOME_VAL','BLUEBOOK','OLDCLAIM']
def float_trans(money):
    try :
        return float(Decimal(re.sub(r'[^\d.]', '', money))) # extracting the float part ( droppign teh dollar sign)
    except:
        return np.nan 

In [17]:
for column in money_features:
    data.loc[:, column] = data.loc[:, column].apply(float_trans)

In [18]:
#results 
data.loc[:, money_features].head()

INCOME  HOME_VAL  BLUEBOOK  OLDCLAIM
0   67349.0       0.0   14230.0    4461.0
1   91449.0  257252.0   14940.0       0.0
2   16039.0  124191.0    4010.0   38690.0
3       0.0  306251.0   15440.0       0.0
4  114986.0  243925.0   18000.0   19217.0

In [19]:
binary_features = ['PARENT1','MSTATUS','RED_CAR','REVOKED','SEX','CAR_USE','URBANICITY']
### differentes values for each featue
print('PARENT1 valeus are ', tuple(data.PARENT1.unique()))
print('MSTATUS valeus are ', tuple(data.MSTATUS.unique()))
print('RED_CAR valeus are ', tuple(data.RED_CAR.unique()))
print('REVOKED valeus are ', tuple(data.REVOKED.unique()))
print('SEX valeus are ', tuple(data.SEX.unique()))
print('CAR_USE valeus are ', tuple(data.CAR_USE.unique()))
print('URBANICITY valeus are ', tuple(data.URBANICITY.unique()))

PARENT1 valeus are  ('No', 'Yes')
MSTATUS valeus are  ('z_No', 'Yes')
RED_CAR valeus are  ('yes', 'no')
REVOKED valeus are  ('No', 'Yes')
SEX valeus are  ('M', 'z_F')
CAR_USE valeus are  ('Private', 'Commercial')
URBANICITY valeus are  ('Highly Urban/ Urban', 'z_Highly Rural/ Rural')


In [20]:
### Converting binary features to 0-1 values
replacing_dict = {'No'  : 0, 'z_No': 0,'no'  : 0, 'Yes' : 1, 'yes' : 1, 'M' : 0,'z_F':1,'Private':0,'Commercial':1,
                 'Highly Urban/ Urban':0,'z_Highly Rural/ Rural':1}
for column in binary_features:
    data.loc[:, column] = data.loc[:, column].replace(replacing_dict)
    

data.loc[:, binary_features].head()

PARENT1  MSTATUS  RED_CAR  REVOKED  SEX  CAR_USE  URBANICITY
0        0        0        1        0    0        0           0
1        0        0        1        0    0        1           0
2        0        1        0        0    1        0           0
3        0        1        1        0    0        0           0
4        0        1        0        1    1        0           0

**Time to deal with the resting features, they are caterorical features, so we should transform them to numerical features, they are two common techniques; one hot encoding and factorization, the second one is used when there is an order between feature values, we see this kind of behaviour in the feature EDUCATION.**

In [21]:
### Numerazing the EDUCATION feature
data.EDUCATION.unique()

array(['PhD', 'z_High School', '<High School', 'Bachelors', 'Masters'],
      dtype=object)

In [22]:
### We consider that  <High School and z_High School are the same thing
data.loc[:, "EDUCATION"].replace({'<High School' : 0,
                                   'z_High School': 0,
                                   'Bachelors'    : 1,
                                   'Masters'      : 2,
                                   'PhD'          : 3}, inplace=True)

**Only 2 features left, we use the one-hot encoding**

In [23]:
object_features  = np.where((data.dtypes != np.float64) & (data.dtypes != np.int64))[0]


In [24]:
object_features  = np.where((data.dtypes != np.float64) & (data.dtypes != np.int64))[0]
encoder = OneHotEncoder(handle_unknown='ignore')
features = encoder.fit_transform(data.iloc[:, object_features]).toarray()

data.drop(columns=data.columns[object_features], inplace=True)
new_columns = []
i = 0
for category in encoder.categories_:
    for new_column in category:
        data[new_column] = features[:, i]
        new_columns.append(new_column)
        i += 1

In [25]:
### Now we have our numerical data
data.dtypes

KIDSDRIV           int64
AGE              float64
HOMEKIDS           int64
YOJ              float64
INCOME           float64
PARENT1            int64
HOME_VAL         float64
MSTATUS            int64
SEX                int64
EDUCATION          int64
TRAVTIME           int64
CAR_USE            int64
BLUEBOOK         float64
TIF                int64
RED_CAR            int64
OLDCLAIM         float64
CLM_FREQ           int64
REVOKED            int64
MVR_PTS            int64
CAR_AGE          float64
URBANICITY         int64
Clerical         float64
Doctor           float64
Home Maker       float64
Lawyer           float64
Manager          float64
Professional     float64
Student          float64
z_Blue Collar    float64
Minivan          float64
Panel Truck      float64
Pickup           float64
Sports Car       float64
Van              float64
z_SUV            float64
dtype: object

**It is time to deal with unbalanced output, we will use a combination of over/under sampling to balance the data, but before that we will save this numerical data because we may use this one for predictions if we don't want to use the data augmentation.**

In [26]:
#### Saving numerical data
new_data_dir = 'numerical_data'
n = len(train)
train_data =  pd.concat([train.target, data.loc[:n-1]], axis=1)
test_data  = pd.concat([test.target, data.loc[n:].reset_index(drop=True)], axis=1)
if not os.path.exists(new_data_dir):
    os.mkdir(new_data_dir)
    
train_data.to_csv(os.path.join(new_data_dir, 'unbalanced_numerical_data.csv'), index=None)
test_data.to_csv(os.path.join(new_data_dir, 'numerical_test_data.csv'), index=None)

### <font color='red'>Data balancing</font>

In [27]:
counter = Counter(train_data.target)
print("of each instance of class 1 we have %f inctance of class 0"% (counter[0]/counter[1]))

of each instance of class 1 we have 2.790525 inctance of class 0


In [28]:
over = SMOTE(sampling_strategy=0.8)
under = RandomUnderSampler(sampling_strategy=1)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
X, y = pipeline.fit_resample(train_data.drop(columns=['target']), train_data.target)

In [29]:
y.value_counts()

1    4806
0    4806
Name: target, dtype: int64

In [30]:
train_data = X.join(y)

In [31]:
train_data.to_csv(os.path.join(new_data_dir,'balanced_numerical_data.csv'), index=None)